## Overview:
Based on GroupSize LaTeX documentation, where we've simplified a lot of notation stuff. Trying to get organized and match notations in documentation.

In [1]:
import numpy as np
import pandas as pd
from pandas import *
import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import csv
import math
from datetime import datetime

In [12]:
# Indices
iset = range(0,41)
jset = {30,60,90,120,180,240,300,360}
demands = {30:2,60:7,90:6,120:8,180:7,240:5,300:2,360:3}
M = 100000
kset = range(1,19)
rset = range(1,7)
gset = [1,2,3,4,5,6]

e = {}
for r in rset:
    e[r] = []
e[1] = [0,1,16,17,34,35,36,37,38,39,40]
e[2] = [0,1,2,3,18,19,36,37,38,39,40]
e[3] = [0,1,2,3,4,5,20,21,38,39,40]
e[4] = [0,1,2,3,4,5,20,21,38,39,40]
e[5] = [0,1,2,3,4,5,6,7,22,23,40]
e[6] = [0,1,2,3,4,5,6,7,22,23,40]

n={}
for i in iset:
    for g in gset:
        n[i] = {g:0}

for i in iset:
    for g in gset:
        if i in e[g]:
            n[i][g] = 0
        else:
            n[i][g] = 1
            
f = {}
for r in rset:
    f[r]=[]
f[1] = [1,2,3]
f[2] = [4,5,6]
f[3] = [7,8,9]
f[4] = [10,11,12]
f[5] = [13,14,15]
f[6] = [16,17,18]

In [13]:
t1 = datetime.now()
env = gp.Env(empty=True)
#env.setParam("OutputFlag",0)
env.start()
m = Model(name="Scheduling -- Group Size 1",env=env,)
adict = demands

# Decision Variables
t = m.addVars(iset, jset, vtype=GRB.INTEGER, name="t")
c = m.addVars(iset, jset, kset, vtype=GRB.BINARY, name="c")   # c=1 if appointment of length j starts at time i in chair k
u = m.addVars(iset, jset, kset, vtype=GRB.INTEGER, name="u")  # number of nurses actively needed at chair k at time i for j length appointments
b = m.addVars(iset, jset, kset, vtype=GRB.BINARY, name="b")   # b=1 if there is an appointment of length j scheduled in chair k at time i

# Violation Variables
vA = m.addVars(iset, vtype=GRB.INTEGER, name="violation A")
vB = m.addVars(iset, vtype=GRB.INTEGER, name="violation B")
vP = m.addVars(iset, vtype=GRB.INTEGER, name="violation pharmacy")
vD = m.addVars(iset, gset, vtype=GRB.INTEGER, name="violation D")            # violation if more than one nurse required
vS = m.addVars(jset, vtype=GRB.INTEGER, name="violation S")      # violation if an appointment is not scheduled

# Cannot schedule appointments that go past 5:00 PM (strict)
c2 = m.addConstrs(c[i,j,k] ==0 for j in jset for i in iset if i>(39-int(j/15)) for k in kset)

# Cannot schedule appointments before 7:30 AM (strict)
c3 = m.addConstr(gp.quicksum(c[0,j,k]+c[1,j,k] for j in jset for k in kset) ==0)

# Pharmacy can do 3 drugs per 30 minutes (add in violation)
c4a = m.addConstrs(gp.quicksum(c[i,j,k]+c[i-1,j,k] for j in jset for k in kset) - vP[i] <= 3 for i in iset if i>=1)
c4b = m.addConstrs(gp.quicksum(c[i,j,k] for j in jset for k in kset) - vP[i] <= 3 for i in iset if i==0)

# Cumulative number of patients in system
c5a = m.addConstrs(t[i,j] == gp.quicksum(c[i-x,j,k] for k in kset for x in range(0,i+1)) for i in iset for j in jset if i+1 < int(j/15))
c5b = m.addConstrs(t[i,j] == gp.quicksum(c[i-x,j,k] for k in kset for x in range(0,int(j/15))) for i in iset for j in jset if i+1 >= int(j/15))

# Times when nurses are actively required at chairs
c6 = m.addConstrs(u[0,j,k] == 0 for j in jset for k in kset)
c7 = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k] for j in jset for i in iset if i<math.floor(29*j/450) if i>0 for k in kset)
c8a = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k]+c[i-math.floor(29*j/450),j,k] for j in jset if j != 30 for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)
c8b = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k] for j in jset if j == 30 for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)

# Nurse busy times (when they cannot be actively required)
c9 = m.addConstrs(gp.quicksum(u[i,j,k] for j in jset for k in f[r]) == 0 for r in rset for i in e[r])

# Violation D: No more than one nurse required at a chair at a time
c10 = m.addConstrs(gp.quicksum(u[i,j,k] for j in jset for k in f[g])-vD[i,g] <= n[i][g] for i in iset for g in gset)

# Violation S: Meet demands
c11 = m.addConstrs(gp.quicksum(c[i,j,k] for i in iset for k in kset)+vS[j] == adict[j] for j in jset)

# Total number of appointments at any time cannot exceed number of chairs
c12 = m.addConstrs(gp.quicksum(t[i,j] for j in jset) <= len(kset) for i in iset)

# Violation A
c13 = m.addConstrs(gp.quicksum(u[i,j,k] for j in jset for k in kset) - gp.quicksum(n[i][g] for g in gset) <= vA[i] for i in iset)

# Violation B
c14 = m.addConstrs(gp.quicksum(t[i,j] for j in jset) <= gp.quicksum(n[i][g] for g in gset)*(3+vB[i]) for i in iset)

# Busy times of chairs
c15 = m.addConstrs(b[0,j,k] == 0 for j in jset for k in kset)
c16 = m.addConstrs(b[i,j,k] == gp.quicksum(c[i-m,j,k] for m in range(0,i+1)) for j in jset for i in iset if i<math.floor(29*j/450) if i>0 for k in kset)
c17 = m.addConstrs(b[i,j,k] == gp.quicksum(c[i-m,j,k] for m in range(0,int(j/15))) for j in jset for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)

# Maximum of one appointment may occur at a time at each chair
c18 = m.addConstrs(gp.quicksum(b[i,j,k] for j in jset) <= 1 for i in iset for k in kset)

# Objective: minimize all violations
obj1 = gp.quicksum(vA[i]+vB[i] for i in iset)
obj2 = gp.quicksum(vP[i] for i in iset)
obj3 = gp.quicksum(vD[i,g] for i in iset for g in gset)
obj4 = gp.quicksum(vS[j]*int(j/15) for j in jset)
obj = obj1 + obj2 + obj3 + obj4
m.setObjective(obj, GRB.MINIMIZE)

m.optimize()
t2 = datetime.now()

runtime = t2 - t1

var_names_violations = []
for var in m.getVars():
    if var.varName[0] == "v":
        if var.X != 0:
            var_names_violations.append(str(var.varName)+str(var.X))

var_names = []
for var in m.getVars():
    if var.X != 0:
        var_names.append(str(var.varName)+str(var.X))
        
var_names_start = []
for var in m.getVars():
    if var.varName[0] == "c":
        if var.X != 0:
            var_names_start.append(str(var.varName)+str(var.X))
            
print(var_names_violations)

Academic license - for non-commercial use only - expires 2023-09-13
Using license file C:\Users\lmdan\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10377 rows, 12513 columns and 118910 nonzeros
Model fingerprint: 0x5ae0bb69
Variable types: 0 continuous, 12513 integer (7872 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 408.0000000
Presolve removed 9503 rows and 10289 columns
Presolve time: 0.84s
Presolved: 874 rows, 2224 columns, 32328 nonzeros
Variable types: 0 continuous, 2224 integer (1784 binary)

Root relaxation: objective 4.933333e+01, 2649 iterations, 0.57 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In [15]:
runtime

datetime.timedelta(seconds=3, microseconds=265653)

In [16]:
var_names_violations

[]

In [39]:
var_names_start

['c[2,300,3]1.0',
 'c[4,300,5]1.0',
 'c[4,240,1]1.0',
 'c[6,180,6]1.0',
 'c[6,180,11]1.0',
 'c[7,240,8]1.0',
 'c[8,360,18]1.0',
 'c[8,120,4]1.0',
 'c[9,120,14]1.0',
 'c[10,120,7]1.0',
 'c[10,90,10]1.0',
 'c[11,60,16]1.0',
 'c[12,360,12]1.0',
 'c[12,360,13]1.0',
 'c[13,60,9]1.0',
 'c[14,240,2]1.0',
 'c[14,180,15]1.0',
 'c[16,90,17]1.0',
 'c[17,180,14]1.0',
 'c[18,30,11]1.0',
 'c[19,240,16]1.0',
 'c[21,180,6]1.0',
 'c[22,240,11]1.0',
 'c[22,180,1]1.0',
 'c[23,120,7]1.0',
 'c[24,90,4]1.0',
 'c[25,180,9]1.0',
 'c[25,120,3]1.0',
 'c[26,60,15]1.0',
 'c[27,120,5]1.0',
 'c[27,60,10]1.0',
 'c[28,90,8]1.0',
 'c[29,120,17]1.0',
 'c[30,90,4]1.0',
 'c[31,120,15]1.0',
 'c[31,90,10]1.0',
 'c[32,60,18]1.0',
 'c[33,60,14]1.0',
 'c[34,60,8]1.0',
 'c[37,30,16]1.0']

In [11]:
# Write to csv
with open('gs1_2.csv', 'w') as myfile:
   wr = csv.writer(myfile, quoting=csv.QUOTE_ALL,lineterminator='\n')
   wr.writerows(zip(var_names_start))

In [ ]:
chairlist = []
for time in v.keys():
    for chair,j in v[time].items():
        if j != 0:
            chairlist.append([time,chair,j])

In [ ]:
with open("chairout_3_8.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(chairlist)